# Customizing Test Output

Want to customize the output of `nbtest`?  There are two key ways to modify the test flow and output:

1. By providing an alternate `untittest.TestRunner` to manage the test cycle. 
1. By providing templates that override the templates in the `templates/` directory.
 
The default templates are designed to provide student centered feedback. If they're not for you you can provide your own Jinja2 templates.

In [1]:
%load_ext nb_unittest

In [2]:
"""@answer2"""

def add(a, b):
    return a + b

The `nbtest.cache.runner_class` variable contains the default runner class. By default it's set to `nbtest.unit.NotebookTestRunner`, which has fewer features than `unittest.TextTestRunner` but provides results more suitable for students. The test code is compatible with other classes in `unittest` so you can replace the runner with your own implementation. 

In [3]:
import unittest
import nb_unittest.tagcache

nb_unittest.tagcache.runner_class = unittest.TextTestRunner

If you like the funky printouts of `unittest.TextTestRunner` you can use it. The `%%testing` magic renders the HTML templates at the end of a run. The HTML templates are compatible with `unittest.TextTestResult`. They just provide less friendly responses.

In [4]:
%%testing @answer2, add

import unittest 

class TestTemplate(unittest.TestCase):

    def test_badd(self):
        """This is a bad test."""
        self.assertEqual(add(1,2), 0, "add(1,2)")

F
FAIL: test_badd (builtins.TestTemplate.test_badd)
This is a bad test.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<testing>", line 8, in test_badd
AssertionError: 3 != 0 : add(1,2)

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (failures=1)


The `nbtest.templ` package contains variables that are used to override the default templates.

In [5]:
# Use your own template by a string.

import jinja2 
from nb_unittest.templ import templ

my_result_template = """
Bulleted list of problems:
<ol>
{% for item in result.failures + result.errors %}
    <li><b>{{ item[0] }}</b><br><i>{{ item[1] }}</i>
{% endfor %}
</ol>
"""

templ.result = jinja2.Environment().from_string(my_result_template)

In [6]:
%%testing @answer2, add

import unittest 

class TestTemplate(unittest.TestCase):

    def test_ok(self):
        """This is a good test."""
        self.assertEqual(add(1,2), 3, "add(1,2)")

    def test_bad1(self):
        """This is a bad test."""
        self.assertEqual(add(1,2), 0, "add(1,2)")
    
    def test_bad2(self):
        """This is a bad test."""
        self.assertEqual(add(1,2), 0, "add(1,2)")

FF.
FAIL: test_bad1 (builtins.TestTemplate.test_bad1)
This is a bad test.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<testing>", line 12, in test_bad1
AssertionError: 3 != 0 : add(1,2)

FAIL: test_bad2 (builtins.TestTemplate.test_bad2)
This is a bad test.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<testing>", line 16, in test_bad2
AssertionError: 3 != 0 : add(1,2)

----------------------------------------------------------------------
Ran 3 tests in 0.001s

FAILED (failures=2)


You can provide an entire Jinja environment if you intend to override *all* of the built in templates:

In [7]:
from jinja2 import DictLoader 

loader = DictLoader({
    "assertion.html": "An assertion has happened: {{ error }}",
    "missing.html": "This attribute is missing: {{ missing }}",
    "wait.html": "Please wait. Beep. Boop.",
    "result.html": my_result_template
})
templ.env = jinja2.Environment(loader=loader)

In [8]:
%%testing @answer2

assert False, "Use assertion template."

In [9]:
%%testing not_here

assert False, "Never get here"

In [10]:
%%testing @answer2 

def test_answer():
    """Test function"""
    assert False, "This is my test"

F
FAIL: unittest.case.FunctionTestCase (test_answer)
Test function
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<testing>", line 4, in test_answer
AssertionError: This is my test

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (failures=1)
